# Introduction to Pytroch

## torch.cuda

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
x = torch.ones(4, 4)

In [ ]:
y = x.cuda()

## torch.autograd

In [ ]:
from torch.autograd import Variable

In [ ]:
x = torch.ones(4, 4)
vx = Variable(x)

In [ ]:
x

In [ ]:
a = Variable(torch.FloatTensor([2]), requires_grad=True)
b = Variable(torch.FloatTensor([3]), requires_grad=True)

In [ ]:
c = a * (b ** 2)

In [ ]:
c.backward(retain_graph=True)

In [ ]:
a.grad

In [ ]:
b.grad

In [ ]:
a.grad = None
c.backward(retain_graph=True)
a.grad, b.grad

In [ ]:
x = torch.ones(4, 4)
vx = Variable(x, requires_grad=True)

In [ ]:
z = vx[0, 0] * vx[0, 1]

In [ ]:
z.backward()

In [ ]:
vx.grad

In [ ]:
x = torch.ones(4, 4)
x[0, 0] = 3

In [ ]:
vx = Variable(x)

vx[0, 0] = 3

In [ ]:
vx = Variable(x, requires_grad=True)

vx[0, 0] = 3

see "In-place operations on Variables" http://pytorch.org/docs/master/autograd.html
to understand why it work when `requires_grad` is set to `False`

In [ ]:
x = torch.ones(4, 4)
vx = Variable(x, requires_grad=True)

In [ ]:
z = vx * vx

In [ ]:
s = torch.sum(z * Variable(torch.ones(4, 4)))
s.backward(retain_graph=True)

In [ ]:
# equivalent to the previous cell
z.backward(torch.ones(4, 4), retain_graph=True)

In [ ]:
vx.grad

## torch.optim

In [ ]:
x = Variable(torch.FloatTensor([3]), requires_grad=True)
y = Variable(torch.FloatTensor([3]), requires_grad=True)

optimizer = torch.optim.SGD([x, y], 0.1)

for _ in range(100):
    optimizer.zero_grad()
    z = x ** 2 + x * y + (y - 2) ** 2
    z.backward()
    optimizer.step()
    
    print("{:8.3}\t{:8.3}\t{:8.3}".format(x.data[0], y.data[0], z.data[0]))

## torch.nn

In [ ]:
import torch

In [ ]:
from torch import nn
from torch.autograd import Variable

In [ ]:
class Perceptron(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.weight = nn.Parameter(torch.randn(30))
    
    def forward(self, x):
        return torch.sum(x * self.weight)


teacher = Perceptron()
student = Perceptron()

optimizer = torch.optim.SGD(student.parameters(), lr=0.01)

for _ in range(200):
    x = Variable(torch.randn(30))
    y_t = teacher(x)
    y_s = student(x)
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = 0.03
    
    optimizer.zero_grad()
    loss = (y_t - y_s) ** 2
    loss.backward()
    optimizer.step()
    
    Q = (student.weight.data * teacher.weight.data).sum() \
        / ((student.weight.data ** 2).sum() * (teacher.weight.data ** 2).sum()) ** 0.5
    print("{:8.3} \t{:8.3} \t{:8.3}".format(y_t.data[0], y_s.data[0], Q))
    

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(teacher.weight.data.numpy())
plt.plot(student.weight.data.numpy())

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear1 = nn.Linear(30, 31, bias=True)
        self.linear2 = nn.Linear(31, 1, bias=False)
    
    def forward(self, x):
        x = self.linear1(x)
        x = nn.functional.relu(x)
        x = self.linear2(x)
        return x

    
teacher = MLP()
student = MLP()

optimizer = torch.optim.SGD(student.parameters(), lr=0.01)

for _ in range(200):
    x = Variable(torch.randn(10, 30))
    y_t = teacher(x)
    y_s = student(x)
        
    optimizer.zero_grad()
    loss = torch.sum((y_t - y_s) ** 2)
    loss.backward()
    optimizer.step()
    
    print("{:8.3} \t{:8.3}".format(y_t.data[0, 0], y_s.data[0, 0]))
    